In [11]:
import numpy as np
import pandas as pd
from twitter_nlp_toolkit.tweet_sentiment_classifier import tweet_sentiment_classifier
from twitter_nlp_toolkit.file_fetcher import file_fetcher
from zipfile import ZipFile
import os

Using this library: https://github.com/eschibli/twitter-toolbox

In [12]:
df = pd.read_csv('./datasets/stacked_clean_again.csv')
df.shape

(1130, 20)

In [13]:
df.head()

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link,day,hour,minute,during_fire,is-fire-related,text_nourl,key_score,from_locations,sent
0,366.0,"Paradise, CA",1.060000e+18,"::paradise baby:: meet jd, the grandson of my ...",2018-11-16 16:31:06+00:00,#campfire,trekcass,@JulieRems,0,0,https://twitter.com/trekcass/status/1063469521...,16,16,31,1,1,"::paradise baby:: meet JD, the grandson of my ...",2,"Paradise, CA",1
1,700.0,"Paradise, CA",1.060000e+18,"¡coño! #campfire @chico, california https://ww...",2018-11-09 19:29:06+00:00,#CampFire,VXO,NaN,0,0,https://twitter.com/VXO/status/106097760184037...,9,19,29,1,1,"¡Coño! #CampFire @Chico, California",1,"Chico, CA",1
2,494.0,"Paradise, CA",1.060000e+18,. . . . . #photography #instapics #photographe...,2018-11-13 15:51:31+00:00,#photography #instapics #photographersofig #pi...,sidewayseightp,NaN,0,0,https://twitter.com/sidewayseightp/status/1062...,13,15,51,1,0,. . . . . #photography #instapics #photographe...,0,"Chico, CA",1
3,53.0,"Paradise, CA",1.070000e+18,.@stucam7771 nailed it! the man in the white h...,2018-11-24 11:29:40+00:00,NaN,XLComedy,@stucam7771,1,0,https://twitter.com/XLComedy/status/1066292766...,24,11,29,1,1,.@Stucam7771 nailed it! The man in the White H...,1,"Paradise, CA",1
4,201.0,"Oroville, CA",1.060000e+18,‘merurica... #voted #uklastudios #uklapictures...,2018-11-07 02:02:51+00:00,#voted #uklastudios #uklapictures,UKLA_Music,NaN,0,0,https://twitter.com/UKLA_Music/status/10599895...,7,2,2,0,0,‘Merurica... #voted #uklastudios #uklapictures...,0,"Gridley, CA",1


In [14]:
chunk = 1 # Fraction of data to train on - you can reduce for debugging for speed
model_path = '.models'

In [15]:
classifier = tweet_sentiment_classifier.SentimentAnalyzer()
classifier.load_small_ensemble()

Model model_full_bow_30k loading...
Model model_full_bow_30k loaded
Model model_bootstrap_glove_50N_200D_30k loading...
Module not found - are you sure it is imported?
Model bert_model_8B_1e-05L_20000.0T_Twitter_0finetuned_1 loading...
Module not found - are you sure it is imported?
Model model_full_glove_30N_200D_30k loading...
Module not found - are you sure it is imported?
Model model_full_glove_10N_100D loading...
Module not found - are you sure it is imported?


small_ensemble.zip: 100%|##########| 469M/469M [00:46<00:00, 10.6MB/s] 


Model model_full_bow_30k loading...
Model model_full_bow_30k loaded
Model model_bootstrap_glove_50N_200D_30k loading...
Module not found - are you sure it is imported?
Model bert_model_8B_1e-05L_20000.0T_Twitter_0finetuned_1 loading...
Module not found - are you sure it is imported?
Model model_full_glove_30N_200D_30k loading...
Module not found - are you sure it is imported?
Model model_full_glove_10N_100D loading...
Module not found - are you sure it is imported?


In [16]:
classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])

array([1., 0., 1., 0.])

In [17]:
df['sent'] = classifier.predict(df['text_nourl'])

In [18]:
df.to_csv('./datasets/stacked_sentiment_again.csv', index=False)

In [ ]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

In [ ]:
df2['sent'].value_counts(normalize=True)

In [ ]:
df2 = df[df['key_score']>0]
df2[['text_nourl', 'sent', 'key_score']]